In [ ]:
"""
CONNECT WITH DRIVE
"""

%pip install gdown
import gdown
import os
import shutil
from google.colab import drive

# Drive
drive.mount('/content/drive')

# url = 'https://drive.google.com/uc?id=1AvXhv8p9ZCUZ2dzbYoA5CP8VT7a8b6Gf' # full
url = 'https://drive.google.com/uc?id=1c4n7GXPn0OTgUsPJwuWWnIignXXJDm6W' # train
url_test = 'https://drive.google.com/uc?id=1PNLYjJ9lGwn2bie2EkVaF1Qw5IcY7rov' # test


output = 'dataset_nonrandom_responses.pth'

gdown.download(url, output, quiet=False)
gdown.download(url_test, f'{output}_test', quiet=False)

# Github
!git clone https://github.com/karolrogozinski/cern_alice_fast_sim_corrvae.git

source = '/content/cern_alice_fast_sim_corrvae'
destination = '/content'

for file in os.listdir(source):
    source_path = os.path.join(source, file)
    dest_path = os.path.join(destination, file)

    try:
        shutil.copy(source_path, dest_path)
    except IsADirectoryError:
        shutil.copytree(source_path, dest_path)


In [ ]:
"""
CorrVAE training based on [TODO]
"""

import time

import numpy as np
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch import optim

from model import ControlVAE
from encoders import EncoderControlVAE
from decoders import DecoderControlVAE

from utils.helpers import plot_epoch, save_model
from utils.loss import get_losses


In [ ]:
"""
PROPERTIES
All of them are normalized

0. - x coordinate of max pixel
1. - y coordinate of max pixel
2. - x coordinate of mass center
3. - y coordinate of mass center
4. - number of non zero pixels
5. - categorized number of non zero pixels
6. - sum of pixels
7. - max pixel value
"""

properties = [2, 3]


In [ ]:
"""
ARGUMENTS
"""

device = 'cpu'

results_dir = '/content/drive/MyDrive/models/'
data_source = 'dataset_nonrandom_responses.pth'

img_size = (1, 44, 44)
latent_dim = 8
latent_dim_prop = 8
latent_dim_cond = 9
hid_channels = 32

num_prop = len(properties)
lr = 1e-4

batch_size = 64
epochs = 250

beta = 1
taus = 0.2
idx_kl = 0
w_kl = 100

lambdas = [
    1000000,
    1,
    1000000,
    1,
    1,
    1
]


In [ ]:
"""
WANDB LOGGER
"""
try:
    import wandb
except:
    %pip install wandb
    import wandb


In [ ]:
token = ...
wandb.login(key=token, relogin=True)


In [ ]:
wandb.init(
    project="CorrVAE_64x64",

    config={
        # "data size": train_size,
        "batch_size": batch_size,
        "beta": beta,
        "taus": taus,
        "loss func": 'sigmoid',
        "lr": lr,
        "num prop": num_prop,
        "latent_dim": latent_dim,
        "latent_dim_prop": latent_dim_prop
    }
)


In [ ]:
"""
DATA
"""

data = torch.load(data_source)
dataset = TensorDataset(data['features'], data['labels'])

train_loader = DataLoader(dataset, shuffle=True, batch_size=batch_size)


data_test = torch.load(f'{data_source}_test')
dataset_test = TensorDataset(data_test['features'], data_test['labels'])

test_loader = DataLoader(dataset_test, shuffle=True, batch_size=batch_size)


In [ ]:
"""
MODEL
"""

encoder = eval("EncoderControlVAE")
decoder = eval("DecoderControlVAE")

model = ControlVAE(img_size, encoder, decoder, latent_dim, latent_dim_prop, latent_dim_cond, num_prop, device=device)
model = model.to(device)
model.train()

optimizer = optim.Adam(model.parameters(), lr=lr)

lr_scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=[50, 100, 150, 200],
        gamma=0.1
)

mse_loss = torch.nn.MSELoss(reduction="sum")

recon_loss_prop_rec = []
recon_loss_rec = []
kl_loss_rec = []
pwwi_loss_rec = []
pwz_loss_rec = []
l1_loss_rec = []
mask_rec = []


In [ ]:
latent_dist_cond_mean = data['labels'][:, 9:].mean(axis=0).to(device)
latent_dist_cond_std = data['labels'][:, 9:].std(axis=0).to(device)


In [ ]:
start_epoch = time.time()

for epoch in range(10, epochs):
    if (epoch + 1) % 10 == 0:
        taus = taus * 0.1

    epoch_loss = []
    epoch_kl_loss = []
    epoch_rec_loss = []
    epoch_prop_loss = []
    epoch_pairwise_loss = []
    epoch_groupwise_loss = []
    epoch_groupwise_cond_loss = []
    epoch_l1_norm = []

    for (data, label) in tqdm(train_loader):
        idx_kl += 1
        data = data.to(device)
        cond = label[:,9:].to(device)

        latent_dist_cond = (
            latent_dist_cond_mean.repeat(data.shape[0], 1),
            latent_dist_cond_std.repeat(data.shape[0], 1)
        )
        latent_sample_cond = label[:, 9:].to(device)

        (reconstruct,y_reconstruct), latent_dist_z, latent_dist_w,\
            latent_sample_z, latent_sample_w, w_mask, cond, mask_ori = model(data, cond, taus)

        latent_sample = torch.cat([latent_sample_w, latent_sample_z], dim=-1)
        latent_dist = (torch.cat([latent_dist_w[0], latent_dist_z[0]], dim=-1),
                       torch.cat([latent_dist_w[1], latent_dist_z[1]], dim=-1))

        ###### Reconstruction loss ######
        rec_loss = F.mse_loss(reconstruct, data, reduction="sum") / 64
        rec_loss = rec_loss / batch_size

        rec_loss_prop = []

        for i, prop in enumerate(properties):
            rec_loss_prop.append(mse_loss(y_reconstruct[:,i], label[:, prop].float().to(device)))

        rec_loss_prop_all = sum(rec_loss_prop)

        ###### Other losses ######
        kl_loss, pairwise_tc_loss, groupwise_tc_loss, groupwise_tc_loss_cond, \
                l1norm, loss, w_kl = get_losses(
            latent_dist,
            latent_sample_w,
            latent_dist_w,
            beta,
            latent_sample_z,
            latent_dist_z,
            w_mask,
            device,
            idx_kl,
            rec_loss,
            rec_loss_prop_all,
            w_kl,
            len(train_loader.dataset),
            lambdas,
            latent_sample_cond,
            latent_dist_cond
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss.append(float(loss))
        epoch_kl_loss.append(float(kl_loss))
        epoch_rec_loss.append(float(rec_loss))
        epoch_prop_loss.append(float(rec_loss_prop_all))
        epoch_pairwise_loss.append(float(pairwise_tc_loss))
        epoch_groupwise_loss.append(float(groupwise_tc_loss))
        epoch_groupwise_cond_loss.append(float(groupwise_tc_loss_cond))
        epoch_l1_norm.append(float(l1norm))

    wandb.log(
        {"total_loss ": np.mean(epoch_loss),
            "KL_loss": np.mean(epoch_kl_loss),
            "rec_loss": np.mean(epoch_rec_loss),
            "rec_prop_loss": np.mean(epoch_prop_loss),
            "wwi_loss": np.mean(epoch_pairwise_loss),
            "wz_loss": np.mean(epoch_groupwise_loss),
            "wc_loss": np.mean(epoch_groupwise_cond_loss),
            "l1_norm": np.mean(epoch_l1_norm),
    })

    save_model(model, optimizer, results_dir, epoch)
    plot_epoch(test_loader, model, device, taus, epoch, time.time() - start_epoch)

    lr_scheduler.step()
    start_epoch = time.time()
